In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/LANL-Earthquake-Prediction"))
print(os.listdir("../input/modeltrain"))

# Any results you write to the current directory are saved as output.

In [ ]:
submission = pd.read_csv('../input/LANL-Earthquake-Prediction/sample_submission.csv', index_col='seg_id', dtype={"time_to_failure": np.float32})
print(submission.shape)
print(submission.head())

In [ ]:
from keras.models import load_model
from keras import backend as K
def cust_maev2(y_true, y_pred):
    return K.mean(K.abs((((y_pred)*16) - ((y_true)*16))), axis=-1)+0.0000001
def cust_mae(y_true, y_pred):
    return K.mean(K.abs((K.exp(y_pred) - K.exp(y_true))), axis=-1)+0.0000001


model = load_model('../input/modeltrain/model.h5',custom_objects={"cust_maev2": cust_maev2,"cust_mae": cust_mae})
model.summary()

In [ ]:
GROUPS=250
windows=[2,3,5,8,13,21,34]
def getsample(input_set):
    final_sample=pd.DataFrame(np.array(input_set).reshape((len(input_set),1)),columns=['acoustic_data'])
    final_sample.index = pd.to_datetime(final_sample.index,unit="s")
    #print("finalsample",final_sample.shape)
    mdata= final_sample.resample(str(GROUPS)+"s").agg(
        #{'acoustic_data' : ['first',max,min,'last','mean',c_std]    
         {'acoustic_data' : ['first',max,min,'last']    
        }    
    )    
    mdata.columns=mdata.columns.droplevel(0)
    #print("mdata",mdata.shape) 

    for window in windows:
        mdata['mafirst{}'.format(window)]=mdata['first'].rolling(window).mean()
        mdata['mamax{}'.format(window)]=mdata['max'].rolling(window).mean()
        mdata['mamin{}'.format(window)]=mdata['min'].rolling(window).mean()
        mdata['malast{}'.format(window)]=mdata['last'].rolling(window).mean()
   #     mdata['mamean{}'.format(window)]=mdata['mean'].rolling(window).mean()
   #     mdata['macstd{}'.format(window)]=mdata['c_std'].rolling(window).mean()                
    mdata=(mdata[windows[-1]:]-5)/3
    #for x in mdata.columns:
     #   mdata[x]=(mdata[x]-mdata[x].min())/(mdata[x].max-mdata[x].min())
    return mdata

In [ ]:
for i, seg_id in enumerate(submission.index):
    seg = pd.read_csv('../input/LANL-Earthquake-Prediction/test/' + seg_id + '.csv')
    x = seg['acoustic_data']
    #print(x.shape)
    sample=getsample(x)
    #print(sample.head())
    sample=np.array(sample)
    
    pred=model.predict(sample.reshape(1,sample.shape[0],sample.shape[1]))
    submission.time_to_failure[i] = pred*16
    #break

submission.to_csv('submission.csv')        
print(submission.head())
print("DONE")